## Get Candlestick Data 

In [1]:
import requests
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
def find_candle_trend (candle_no):
    if candle_no < 10:
        return 0
    
    data = pd.read_csv(filename)
    data = data.iloc[candle_no-5:candle_no+1]
#    display (data[['Open','Close','Low','High']])
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = int(r2_score(Y, y_pred)*100)
#    print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)
    
    if coeficient > 0:
        Current_Market = 1 *  Current_Market_Fit## Bullish / Buy ##
    else:
        Current_Market = -1 * Current_Market_Fit ## Bearish / Sell ##
    
    return Current_Market

In [5]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    instrument = cfg['currency']['instrument']

In [6]:
print('Instrument:', instrument)

Instrument: EUR_USD


### The Configs for Run:

In [7]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [8]:
Load_10K_Records=True
Load_20K_Records=False

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',

currency_pairs = [instrument]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [9]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [10]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [11]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [12]:
if Load_10K_Records:
    
    #### Load 10K data
        
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
    datetime_object_15K = parser.parse(second_response['candles'][0]['time'])
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    
    if Load_20K_Records:

        #### Load 15K data

        date= datetime_object_15K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        third_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(third_response['candles'][0]['time'])
        third_response= third_response['candles']
        third_response.extend(second_response)

        #### Load 20K data

        date= datetime_object_20K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        fourth_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(fourth_response['candles'][0]['time'])
        fourth_response= fourth_response['candles']
        fourth_response.extend(third_response)

        response=fourth_response
    else:
        response=second_response 
else:
    response=first_response['candles']

In [13]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [15]:
data.shape

(9658, 12)

In [16]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-12-04,02:00:00,2,338,4,Friday,2015-12-04T02:00:00.000000000Z,3762,1.09104,1.09464,1.09046,1.09390
1,2015-12-04,06:00:00,6,338,4,Friday,2015-12-04T06:00:00.000000000Z,10055,1.09388,1.09452,1.08587,1.08899
2,2015-12-04,10:00:00,10,338,4,Friday,2015-12-04T10:00:00.000000000Z,14955,1.08901,1.09507,1.08555,1.09258
3,2015-12-04,14:00:00,14,338,4,Friday,2015-12-04T14:00:00.000000000Z,18128,1.09256,1.09508,1.08363,1.08684
4,2015-12-04,18:00:00,18,338,4,Friday,2015-12-04T18:00:00.000000000Z,5358,1.08680,1.08851,1.08512,1.08823


In [17]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9653,2022-02-18,10:00:00,10,49,4,Friday,2022-02-18T10:00:00.000000000Z,29630,1.13710,1.13725,1.13468,1.13490
9654,2022-02-18,14:00:00,14,49,4,Friday,2022-02-18T14:00:00.000000000Z,48016,1.13491,1.13542,1.13140,1.13224
9655,2022-02-18,18:00:00,18,49,4,Friday,2022-02-18T18:00:00.000000000Z,15439,1.13224,1.13358,1.13219,1.13237
9656,2022-02-20,22:00:00,22,51,6,Sunday,2022-02-20T22:00:00.000000000Z,15695,1.13221,1.13580,1.13132,1.13536
9657,2022-02-21,02:00:00,2,52,0,Monday,2022-02-21T02:00:00.000000000Z,1274,1.13534,1.13568,1.13530,1.13553


## Simple Moving Average (SMA)

In [18]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [19]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [20]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [21]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
9653,2022-02-18,10:00:00,10,49,4,Friday,2022-02-18T10:00:00.000000000Z,29630,1.13710,1.13725,1.13468,1.13490,1.1363,1.1363,1.1362,-0.00140,-0.00140,-0.00130
9654,2022-02-18,14:00:00,14,49,4,Friday,2022-02-18T14:00:00.000000000Z,48016,1.13491,1.13542,1.13140,1.13224,1.1355,1.1358,1.1361,-0.00326,-0.00356,-0.00386
9655,2022-02-18,18:00:00,18,49,4,Friday,2022-02-18T18:00:00.000000000Z,15439,1.13224,1.13358,1.13219,1.13237,1.1346,1.1354,1.1360,-0.00223,-0.00303,-0.00363
9656,2022-02-20,22:00:00,22,51,6,Sunday,2022-02-20T22:00:00.000000000Z,15695,1.13221,1.13580,1.13132,1.13536,1.1344,1.1353,1.1359,0.00096,0.00006,-0.00054
9657,2022-02-21,02:00:00,2,52,0,Monday,2022-02-21T02:00:00.000000000Z,1274,1.13534,1.13568,1.13530,1.13553,1.1341,1.1353,1.1359,0.00143,0.00023,-0.00037


## Price Range

In [22]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [23]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [24]:
%%time
for candle_no in data.index.tolist():
#    print (candle_no)
    data.at[candle_no, "Trend"] = find_candle_trend(candle_no)

CPU times: user 7min 20s, sys: 9.24 s, total: 7min 29s
Wall time: 7min 29s


In [25]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5,Trend
9634,2022-02-18,10:00:00,10,49,4,Friday,2022-02-18T10:00:00.000000000Z,29630,1.13710,1.13725,...,0.00257,0.00235,-0.00022,0,0.00220,0.00235,0.00242,-0.00050,0.00023,-3.0
9635,2022-02-18,14:00:00,14,49,4,Friday,2022-02-18T14:00:00.000000000Z,48016,1.13491,1.13542,...,0.00402,0.00318,-0.00084,0,0.00267,0.00318,0.00351,0.00220,-0.00050,-53.0
9636,2022-02-18,18:00:00,18,49,4,Friday,2022-02-18T18:00:00.000000000Z,15439,1.13224,1.13358,...,0.00139,0.00121,-0.00018,1,-0.00013,0.00121,0.00005,0.00267,0.00220,-80.0
9637,2022-02-20,22:00:00,22,51,6,Sunday,2022-02-20T22:00:00.000000000Z,15695,1.13221,1.13580,...,0.00448,0.00044,-0.00404,1,-0.00315,0.00044,0.00089,-0.00013,0.00267,-35.0
9638,2022-02-21,02:00:00,2,52,0,Monday,2022-02-21T02:00:00.000000000Z,1274,1.13534,1.13568,...,0.00038,0.00015,-0.00023,1,-0.00019,0.00015,0.00004,-0.00315,-0.00013,-3.0


In [26]:
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)